In [ ]:
using Random

pwd()

In [2]:
function recup_numbre_dans_string(str::String,i::Int) #cette fonction recupere le nombre qui commence à l'indice i dans le string str
    j = i
    while length(str) >= j+1 && str[j+1] >= '0' && str[j+1] <= '9' #verifie si on manipule bien un chiffre
        j+=1
    end
    return (parse(Int64,str[i:j]),j)
end

recup_numbre_dans_string (generic function with 1 method)

In [3]:
struct Instance
    graphe::BitMatrix # Matrice d'adjacence du graphe
    k::Int
end

mutable struct Solution
    nodecolors::Vector{Int}
end

"""Renvoie le nombre de sommets du graphe de l'instance."""
function Base.length(instance::Instance)
    return size(instance.graphe)[1]
end

function Base.length(solution::Solution)
    return length(solution.nodecolors)
end

In [4]:
function read_instance(path::String,k::Int)
    io = open(path)
    b = true
    graphe = BitMatrix
    while (b)
        ligne = readline(io)
        if length(ligne) >= 1 && ligne[1] == 'p'
            n = recup_numbre_dans_string(ligne,8)[1] #nombre de sommets dans le graphe
            graphe = falses(n,n)
        end
        if length(ligne) >= 1 && ligne[1] == 'e'
            u,j = recup_numbre_dans_string(ligne,3)
            v = recup_numbre_dans_string(ligne,j+1)[1]
            graphe[u,v] = true
            graphe[v,u] = true
        end
        if length(ligne) < 1
            b = false
        end
    end
    return Instance(graphe,k)
end

read_instance (generic function with 1 method)

In [5]:
begin
    instance = read_instance("graphs/dsjc125.1.col")
    display(instance.graphe)
end

MethodError: MethodError: no method matching read_instance(::String)

Closest candidates are:
  read_instance(::String, !Matched::Int64)
   @ Main c:\Users\SENEGAS\Documents\ENPC\3A\Proj-meta\Jupyter tests.ipynb:1


In [6]:
"""Détermine si les deux sommets indiqués ont la même couleur."""
function is_collision(instance::Instance,solution::Solution,i::Int,j::Int)
    return instance.graphe[i,j] && solution.nodecolors[i] == solution.nodecolors[j]
end

is_collision

In [7]:
"""Calcule le nombre de collisions de la coloration."""
function nbr_collision(instance::Instance,solution::Solution)
    compteur = 0 
    for i = 1:length(instance)
        for j = i+1:length(instance)
            compteur += is_collision(instance,solution,i,j)
        end
    end
    return compteur
end

nbr_collision

Les couleurs sont entre 1 et k

In [8]:
"""
Calcule le nombre de collisions au sommet position en lui attribuant couleur.
"""
function nb_collision_sommet_couleur(instance::Instance,solution::Solution,position::Int,couleur::Int) 
    n_collision = 0
    for  i = 1:length(solution)
        n_collision += instance.graphe[position,i] && solution.nodecolors[i] == couleur
    end
    return n_collision
end

nb_collision_sommet_couleur

In [9]:
"""Calcule l'ordre dans lequel traiter les sommets dans l'heuristique gloutonne (degré décroissant)."""
function calcul_de_ordre_des_sommets(instance::Instance) 
    deg = vec(sum(instance.graphe,dims=1))
    return sortperm(deg,rev=true)
end

"""Détermine la meilleure couleur à attribuer au nœud."""
function meilleure_couleur_locale(instance::Instance,solution::Solution,position::Int)
    best_c = 1
    best_nb_collision = length(solution)
    for c = 2:instance.k
        nb_collision = nb_collision_sommet_couleur(instance,solution,position,c)
        if nb_collision < best_nb_collision
            best_c = c
            best_nb_collision = nb_collision
        end
    end
    return best_c
end

meilleure_couleur_locale

In [10]:
"""Heuristique gloutonne générant une solution par coloration successive des sommets."""
function glouton(instance::Instance)
    solution = Solution(zeros(Int,length(instance))) # la couleur 0 veut dire non colorée
    ordre_des_sommets = calcul_de_ordre_des_sommets(instance)
    for i ∈ ordre_des_sommets
        c = meilleure_couleur_locale(instance,solution,i)
        solution.nodecolors[i] = c
    end
    return solution
end

glouton

In [11]:
begin
    k = 5
    instance = read_instance("graphs/dsjc125.1.col",k)
    aleatoire = Solution(rand(1:k,125))

    println("SOLUTION ALÉATOIRE:")
    println(aleatoire)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

SOLUTION ALÉATOIRE:


Solution([5, 4, 2, 2, 3, 1, 3, 1, 5, 5, 2, 4, 3, 4, 4, 1, 4, 2, 2, 3, 3, 5, 1, 4, 4, 5, 3, 2, 4, 1, 1, 2, 4, 2, 2, 1, 3, 3, 4, 3, 1, 3, 5, 1, 5, 3, 3, 1, 5, 1, 5, 2, 1, 5, 5, 2, 1, 2, 5, 1, 5, 3, 4, 1, 4, 5, 1, 5, 1, 1, 1, 5, 5, 5, 3, 4, 2, 3, 4, 1, 3, 2, 3, 2, 3, 1, 5, 1, 2, 5, 4, 1, 3, 1, 2, 2, 2, 3, 2, 1, 1, 3, 2, 5, 5, 2, 2, 3, 1, 2, 2, 3, 2, 1, 5, 3, 1, 2, 4, 4, 5, 4, 5, 5, 1])


collisions: 150



SOLUTION GLOUTONNE:
Solution([3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3])
collisions: 47
